In [ ]:
# rag_ollama_chroma.py
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os
from pathlib import Path

In [ ]:
root_path = Path(os.getcwd()).parent.parent
ollama_model = root_path / "ollama-service" / "models" / "ko_llama" / "llama-3-Korean-Bllossom-8B.Q4_K_M.gguf"
embeddings_path = root_path / "ollama-service" / "models" / "BGE-m3-ko"

embedding_model = HuggingFaceEmbeddings(
    model_name=str(embeddings_path)
)
vectorstore = FAISS.load_local("vector_db/pcn_web2", embedding_model, allow_dangerous_deserialization=True)


retriever = vectorstore.as_retriever(search_kwargs={"k": 20})  # "score_threshold": 0.75
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


llm = ChatOllama(
    model=str("ko-llama-8B"),
    temperature=0.1,  # 약간의 다양성 부여
    base_url="http://localhost:11434",
    callback_manager=callback_manager,
    retriever=retriever  # retriever를 llm에 직접 설정
)

# 프롬프트를 더 명확하고 자연스럽게 개선
system_template = (
    '''
    당신은 (주)피씨엔(PCN)의 친절하고 유능한 AI 어시스턴트입니다.\n
    사용자의 질문에 대해 신뢰할 수 있는 정보를 바탕으로 정확하고 간결하게 답변하세요.\n
    주어진 정보를 바탕으로 답변하세요.\n
    **피씨엔**, 또는 **PCN**이라는 정보를 입력받으면 피씨엔(기술), PCN 회사 소개를 답변하세요.\n
    피씨엔, PCN 회사 소개는 아래 내용을 기반으로 답변하세요.\n
    피씨엔(PCN)은 기술 회사로 Bigdata, XR, AI, SI 등 다양한 서비스를 제공합니다.\n
    회사개요, 프로젝트(Project), 주요 솔루션, 조직규모, 주요 고객, 연혁 등 회사 정보를 답변하세요.\n
    프로젝트 설명 시, 프로젝트 이름, 프로젝트 설명, 프로젝트 결과 등을 1~2문장으로 간단하게 답변하세요.\n
    답변에서 회사명은 PCN으로 표기하세요.\n
    '''
)
human_template = (
    """
    주어진 질문은 사실에 근거하여 답변하세요.\n
    답변은 한글로 작성하세요.\n
    답변은 주어진 질문에 대해 단계별로 논리적으로 생각하여 답변해 주세요.\n
    질문: {question}
    """
)

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = prompt | llm



In [ ]:
# result1 = chain.invoke({"question": "피씨엔 회사 소개"})

# Eval

In [ ]:
from langchain.evaluation.qa import QAEvalChain

# 평가용 질문-정답 쌍 예시
eval_data = [
    {
        "query": "PCN의 주요 서비스는 무엇인가요?",
        "answer": "PCN은 빅데이터, XR, AI, SI 등 다양한 기술 서비스를 제공합니다."
    },
    {
        "query": "PCN의 설립 연도는 언제인가요?",
        "answer": "PCN은 2015년에 설립되었습니다."
    },
    {
        "query": "PCN의 주요 고객은 어떤 산업 분야에 있나요?",
        "answer": "PCN의 주요 고객은 금융, 제조, 유통, 교육 등 다양한 산업 분야에 있습니다."
    }
]
predictions = []
for item in eval_data:
    try:
        result = chain.invoke({"question": item["query"]})
        predictions.append(str(result))
        print(f"질문 처리 완료: {item['query'][:30]}...")
    except Exception as e:
        print(f"질문 처리 실패: {item['query']} - 에러: {e}")
        predictions.append("")  # 빈 문자열로 채움
print(f"eval_data 길이: {len(eval_data)}")
print(f"predictions 길이: {len(predictions)}")


In [ ]:
import evaluate

# 정답과 예측이 완전히 일치하면 1, 아니면 0으로 처리
references = []
preds = []

for item, pred in zip(eval_data, predictions):
    if item["answer"].strip() == pred.strip():
        references.append(1)
        preds.append(1)
    else:
        references.append(1)
        preds.append(0)

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

# evaluate 라이브러리는 zero_division 매개변수를 지원하지 않으므로 제거
accuracy = accuracy_metric.compute(references=references, predictions=preds)
precision = precision_metric.compute(references=references, predictions=preds, average="binary")
recall = recall_metric.compute(references=references, predictions=preds, average="binary")
f1 = f1_metric.compute(references=references, predictions=preds, average="binary")

print("=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===")
print(f"정확도(Accuracy): {accuracy['accuracy']:.2f}")
print(f"정밀도(Precision): {precision['precision']:.2f}")
print(f"재현율(Recall): {recall['recall']:.2f}")
print(f"F1 점수(F1 Score): {f1['f1']:.2f}")